## Data Simulation for future covariates to predict energy prices 


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from darts import TimeSeries
from darts.models import ExponentialSmoothing


In [9]:
# Import the data 
def load_and_prepare_data(file_path):
    """
    Load energy prices data from a CSV file, ensure chronological order, and convert 'Date' to datetime.
    """
    df = pd.read_csv(file_path)
    df.sort_values('Date', inplace=True)
    df.set_index('Date', inplace=True)
    df = pd.DataFrame(df)
    return df

In [10]:
# Import the data
df = load_and_prepare_data('../../data/Final_data/final_data_july.csv')

# Reset the index
df.reset_index(inplace=True)
# Ensure the 'Date' column is properly parsed and set as the index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Ensure the data has a daily frequency (Filling missing dates if necessary)
df = df.asfreq('D')  # This makes sure the data has a daily frequency

In [11]:
df

,Day_ahead_price (€/MWh),Solar_radiation (W/m2),Wind_speed (m/s),Temperature (°C),Biomass (GWh),Hard_coal (GWh),Hydro (GWh),Lignite (GWh),Natural_gas (GWh),Other (GWh),Pumped_storage_generation (GWh),Solar_energy (GWh),Wind_offshore (GWh),Wind_onshore (GWh),Net_total_export_import (GWh),BEV_vehicles,Oil_price (EUR),TTF_gas_price (€/MWh),Nuclear_energy (GWh)
Date,,,,,,,,,,,,,,,,,,,
2012-01-01,18.19,14.75,4.95,8.39,98.605,108.454,51.011,325.337,188.811,54.040,19.314,6.263,3.404,235.467,54.662,6,99.64,21.10,250.979
2012-01-02,33.82,15.12,5.00,7.41,98.605,222.656,51.862,343.168,229.293,54.166,28.892,6.312,3.350,231.772,-64.477,6,100.04,20.00,258.671
2012-01-03,35.03,31.88,7.77,5.23,98.605,162.204,48.851,336.773,241.297,53.518,21.072,24.226,7.292,504.484,-35.078,6,100.44,20.90,271.495
2012-01-04,32.16,25.21,8.04,4.78,98.605,189.633,47.101,323.976,252.289,52.194,28.300,14.157,7.828,541.528,22.924,6,103.15,21.40,270.613
2012-01-05,20.35,13.46,9.98,4.23,98.605,175.733,45.854,327.502,259.018,52.179,31.887,4.728,8.280,572.819,35.618,6,103.92,21.30,287.555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-24,66.61,225.04,3.47,17.54,110.007,43.469,85.857,199.246,194.291,54.026,20.934,325.285,49.360,179.921,-168.705,992,75.75,32.63,0.000
2024-07-25,78.34,272.71,2.12,17.85,110.410,50.676,82.632,195.983,209.610,52.963,18.766,394.116,51.053,42.885,-194.496,992,76.36,31.70,0.000
2024-07-26,93.04,172.33,2.60,19.09,110.852,42.333,79.531,205.273,205.773,52.616,19.081,256.246,40.449,129.267,-241.786,993,75.21,32.20,0.000


# Hyperparametertuning

In [12]:
# Install needed packages
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly, plot_cross_validation_metric
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import itertools
from prophet.diagnostics import cross_validation, performance_metrics
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import time

In [13]:
# Import the data
df = load_and_prepare_data('../../data/Final_data/final_data.csv')

# Reset the index
df = df.reset_index()

#rename date and target column to ds and y
df = df.rename(columns={'Date': 'ds', 'Hydro (GWh)': 'y'})

In [14]:
# Define the parameter grid
param_dist = {  
    'changepoint_prior_scale': [0.01, 0.05, 0.1, 0.25, 0.5],
    'seasonality_prior_scale': [1, 5, 7, 10, 12],
    'holidays_prior_scale': [1, 5, 7, 10, 12],
    'seasonality_mode': ['additive', 'multiplicative'],
}


# Generate all combinations of parameters
all_params = [dict(zip(param_dist.keys(), v)) for v in itertools.product(*param_dist.values())]

# Function to run Prophet with a specific parameter set
def evaluate_params(params):
    try:
        # Fit model with given params
        m = Prophet(**params).fit(df)
        df_cv = cross_validation(m, initial='730 days', period='180 days', horizon='365 days')
        df_p = performance_metrics(df_cv, rolling_window=0.1)

        # Calculate the average RMSE across all horizons
        avg_rmse = df_p['rmse'].mean()

        return params, avg_rmse
    except Exception as e:
        # Handle any exceptions that occur during evaluation
        print(f"Error occurred with parameters {params}: {e}")
        return params, float('inf'), pd.Timedelta(seconds=0)

# Start the timer
start_time = time.time()

# Use ThreadPoolExecutor to parallelize the evaluation
with ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(evaluate_params, all_params), total=len(all_params)))

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"Total execution time: {elapsed_time:.2f} seconds")

# Extract the RMSEs, horizons, and corresponding params
rmses = [result[1] for result in results]
tuning_results = pd.DataFrame([result[0] for result in results])
tuning_results['average_rmse'] = rmses

# Output the tuning results
print(tuning_results)


  0%|          | 0/250 [00:00<?, ?it/s]10:13:02 - cmdstanpy - INFO - Chain [1] start processing
10:13:02 - cmdstanpy - INFO - Chain [1] start processing
10:13:02 - cmdstanpy - INFO - Chain [1] done processing
10:13:02 - cmdstanpy - INFO - Chain [1] start processing
10:13:02 - cmdstanpy - INFO - Chain [1] start processing
10:13:02 - cmdstanpy - INFO - Chain [1] start processing
10:13:02 - cmdstanpy - INFO - Chain [1] start processing
10:13:02 - cmdstanpy - INFO - Chain [1] start processing
10:13:02 - cmdstanpy - INFO - Chain [1] start processing
10:13:02 - cmdstanpy - INFO - Chain [1] start processing
10:13:02 - cmdstanpy - INFO - Chain [1] done processing
10:13:02 - cmdstanpy - INFO - Chain [1] start processing
10:13:02 - cmdstanpy - INFO - Chain [1] start processing
10:13:02 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/19 [00:00<?, ?it/s]

10:13:02 - cmdstanpy - INFO - Chain [1] done processing
10:13:02 - cmdstanpy - INFO - Chain [1] done processing
10:13:02 - cmdstanpy - INFO - Chain [1] done processing
10:13:02 - cmdstanpy - INFO - Chain [1] start processing


  0%|          | 0/19 [00:00<?, ?it/s]

10:13:03 - cmdstanpy - INFO - Chain [1] done processing


...

  0%|          | 0/19 [00:00<?, ?it/s]

10:44:28 - cmdstanpy - INFO - Chain [1] done processing
10:44:28 - cmdstanpy - INFO - Chain [1] start processing
10:44:28 - cmdstanpy - INFO - Chain [1] start processing
10:44:28 - cmdstanpy - INFO - Chain [1] done processing
10:44:29 - cmdstanpy - INFO - Chain [1] done processing
10:44:29 - cmdstanpy - INFO - Chain [1] start processing
10:44:29 - cmdstanpy - INFO - Chain [1] done processing
10:44:30 - cmdstanpy - INFO - Chain [1] start processing
10:44:30 - cmdstanpy - INFO - Chain [1] start processing
10:44:30 - cmdstanpy - INFO - Chain [1] done processing
10:44:30 - cmdstanpy - INFO - Chain [1] done processing
10:44:31 - cmdstanpy - INFO - Chain [1] start processing
10:44:31 - cmdstanpy - INFO - Chain [1] start processing
10:44:31 - cmdstanpy - INFO - Chain [1] done processing
10:44:31 - cmdstanpy - INFO - Chain [1] start processing


  0%|          | 0/19 [00:00<?, ?it/s]

10:44:32 - cmdstanpy - INFO - Chain [1] start processing
10:44:32 - cmdstanpy - INFO - Chain [1] start processing
10:44:32 - cmdstanpy - INFO - Chain [1] start processing
10:44:32 - cmdstanpy - INFO - Chain [1] done processing
10:44:33 - cmdstanpy - INFO - Chain [1] start processing
10:44:33 - cmdstanpy - INFO - Chain [1] done processing
10:44:33 - cmdstanpy - INFO - Chain [1] start processing
10:44:33 - cmdstanpy - INFO - Chain [1] start processing
10:44:34 - cmdstanpy - INFO - Chain [1] done processing
10:44:34 - cmdstanpy - INFO - Chain [1] done processing
10:44:34 - cmdstanpy - INFO - Chain [1] done processing
10:44:35 - cmdstanpy - INFO - Chain [1] done processing
10:44:35 - cmdstanpy - INFO - Chain [1] done processing
10:44:35 - cmdstanpy - INFO - Chain [1] done processing
10:44:35 - cmdstanpy - INFO - Chain [1] start processing
10:44:35 - cmdstanpy - INFO - Chain [1] start processing
10:44:36 - cmdstanpy - INFO - Chain [1] start processing
10:44:36 - cmdstanpy - INFO - Chain [1]

Total execution time: 1960.96 seconds
     changepoint_prior_scale  seasonality_prior_scale  holidays_prior_scale  \
0                       0.01                        1                     1   
1                       0.01                        1                     1   
2                       0.01                        1                     5   
3                       0.01                        1                     5   
4                       0.01                        1                     7   
..                       ...                      ...                   ...   
245                     0.50                       12                     7   
246                     0.50                       12                    10   
247                     0.50                       12                    10   
248                     0.50                       12                    12   
249                     0.50                       12                    12   

    seasonali

In [15]:
# Python
best_params = all_params[np.argmin(rmses)]
print(best_params)


{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 1, 'holidays_prior_scale': 1, 'seasonality_mode': 'additive'}


## Applying the best parameter configuration to the model

In [20]:
# Import the data
df = load_and_prepare_data('../../data/Final_data/final_data_july.csv')

# Reset the index
df.reset_index(inplace=True)
# Ensure the 'Date' column is properly parsed and set as the index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Ensure the data has a daily frequency 
df = df.asfreq('D') 

In [21]:
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly
import plotly.graph_objs as go

# Load your data (adjust the path and data columns)
# Prepare data for Prophet
df_prophet = pd.DataFrame({
    'ds': df.index, 
    'y': df['Hydro (GWh)']
})

# Initialize the Prophet model
m = Prophet(
    seasonality_mode='additive',
    changepoint_prior_scale = 0.01,
    holidays_prior_scale=1, 
    seasonality_prior_scale=1, 
    yearly_seasonality=20,
    weekly_seasonality=True,  
    daily_seasonality=False,
    scaling="absmax", 
    interval_width=0.9,   
)

# Add holidays for Germany to the data 
m.add_country_holidays(country_name='DE')

# Fit the model
m.fit(df_prophet)

# Make future predictions for the next 2 years (730 days)
future = m.make_future_dataframe(periods=730)
forecast = m.predict(future)

# Use Plotly for visualization 
fig = plot_plotly(m, forecast)

# Add additional layout elements if necessary
fig.update_layout(
    title="Forecasted Hydro Electricity Generation",
    xaxis_title="Date",
    yaxis_title="Hydro Electricity Generation (GWh)",
    legend=dict(
        x=1, y=1,
        xanchor="right",
        yanchor="top"
    ),
    template="plotly_white"
)

# Show the Plotly plot
fig.show()


10:58:16 - cmdstanpy - INFO - Chain [1] start processing
10:58:16 - cmdstanpy - INFO - Chain [1] done processing


In [22]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Ascension Day,Ascension Day_lower,Ascension Day_upper,Christmas Day,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2012-01-01,59.264651,25.448738,60.923162,59.264651,59.264651,0.0,0.0,0.0,0.0,...,-2.218140,-2.218140,-2.218140,-10.097964,-10.097964,-10.097964,0.0,0.0,0.0,43.961154
1,2012-01-02,59.260937,30.545787,66.415909,59.260937,59.260937,0.0,0.0,0.0,0.0,...,-1.071756,-1.071756,-1.071756,-9.948468,-9.948468,-9.948468,0.0,0.0,0.0,48.240713
2,2012-01-03,59.257224,32.682066,68.205371,59.257224,59.257224,0.0,0.0,0.0,0.0,...,0.887058,0.887058,0.887058,-9.668332,-9.668332,-9.668332,0.0,0.0,0.0,50.475949
3,2012-01-04,59.253510,33.940629,69.858359,59.253510,59.253510,0.0,0.0,0.0,0.0,...,1.245054,1.245054,1.245054,-9.259037,-9.259037,-9.259037,0.0,0.0,0.0,51.239527
4,2012-01-05,59.249797,34.694502,69.202464,59.249797,59.249797,0.0,0.0,0.0,0.0,...,0.946316,0.946316,0.946316,-8.733334,-8.733334,-8.733334,0.0,0.0,0.0,51.462779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5318,2026-07-24,59.610519,51.023597,88.257841,58.887495,60.255268,0.0,0.0,0.0,0.0,...,0.670529,0.670529,0.670529,9.771635,9.771635,9.771635,0.0,0.0,0.0,70.052682
5319,2026-07-25,59.615286,51.266906,86.519588,58.890285,60.261660,0.0,0.0,0.0,0.0,...,-0.459060,-0.459060,-0.459060,9.730399,9.730399,9.730399,0.0,0.0,0.0,68.886625
5320,2026-07-26,59.620053,49.074541,85.005785,58.893075,60.268053,0.0,0.0,0.0,0.0,...,-2.218140,-2.218140,-2.218140,9.616699,9.616699,9.616699,0.0,0.0,0.0,67.018612
5321,2026-07-27,59.624820,49.304137,84.746592,58.895865,60.274446,0.0,0.0,0.0,0.0,...,-1.071756,-1.071756,-1.071756,9.435929,9.435929,9.435929,0.0,0.0,0.0,67.988993


In [23]:
# select column ds and yhat into a new dataframe and create a copy to avoid the warning
forecast_final = forecast[['ds', 'yhat']].copy()

# rename ds to Date and yhat to Temperature (°C)
forecast_final.rename(columns={'ds': 'Date', 'yhat': 'Hydro (GWh)'}, inplace=True)

# reduce the number of decimal places
forecast_final = forecast_final.round(2)

# set Date as index
forecast_final.set_index('Date', inplace=True)

# Select the data after the 28.07.2024
forecast_final = forecast_final.loc['2024-07-29':]

# save the results to a csv file
forecast_final.to_csv('../../data/Future_data/forecasted_hydro.csv')
